In [8]:
import random
import numpy as np

class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.zeros([y, 1]) for y in sizes[1:]]
        self.weights = [np.zeros([y, x])
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        n = len(training_data)
        for j in xrange(epochs):
        # random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)


    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]
        self.nabla_b = nabla_b
        self.nabla_w = nabla_w
        
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        self.activations = activations
        self.zs = zs

        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [9]:
import mnist_loader

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [98]:
n=1000
epochs = 30
hidden = 10

In [99]:
data = training_data[:n]
net = Network([784, hidden, 10])

In [100]:
#%%timeit 
net.SGD(data, epochs=epochs, mini_batch_size=n, eta=3.0)

In [101]:
net.weights[1]

array([[-0.22113814, -0.22113814, -0.22113814, -0.22113814, -0.22113814,
        -0.22113814, -0.22113814, -0.22113814, -0.22113814, -0.22113814],
       [-0.20590532, -0.20590532, -0.20590532, -0.20590532, -0.20590532,
        -0.20590532, -0.20590532, -0.20590532, -0.20590532, -0.20590532],
       [-0.21865228, -0.21865228, -0.21865228, -0.21865228, -0.21865228,
        -0.21865228, -0.21865228, -0.21865228, -0.21865228, -0.21865228],
       [-0.22631396, -0.22631396, -0.22631396, -0.22631396, -0.22631396,
        -0.22631396, -0.22631396, -0.22631396, -0.22631396, -0.22631396],
       [-0.21259947, -0.21259947, -0.21259947, -0.21259947, -0.21259947,
        -0.21259947, -0.21259947, -0.21259947, -0.21259947, -0.21259947],
       [-0.22743271, -0.22743271, -0.22743271, -0.22743271, -0.22743271,
        -0.22743271, -0.22743271, -0.22743271, -0.22743271, -0.22743271],
       [-0.22506704, -0.22506704, -0.22506704, -0.22506704, -0.22506704,
        -0.22506704, -0.22506704, -0.22506704